In [1]:
from django.db.models import F, ExpressionWrapper, fields
import math
import csv

In [2]:
from datetime import datetime, timedelta
duration_cal = ExpressionWrapper(F('stop') - F('start'), output_field=fields.DurationField())
r = Rent.objects.all().filter(~Q(stop = None)).annotate(duration=duration_cal)

for i in r:
    if(i.cost == None):
        hour = math.floor(i.duration.seconds/3600)
        cost_cal = i.room.hour_price*hour
        i.cost=cost_cal
        i.save()

In [3]:
csvfile = "5709611957.csv"
members = Member.objects.all()
with open(csvfile, "w") as output:
    fieldnames = ['id', 'cost']
    writer = csv.DictWriter(output, fieldnames=fieldnames, lineterminator='\n')
    writer.writeheader()
    for u in members:
        u_price = r.filter(member=u).aggregate(Sum('cost'))
        data = {}
        data['id'] = u.id
        data['cost'] =u_price['cost__sum']
        writer.writerow(data)  